## EDA Part 1

### Load credentials from Project root

In [1]:
import os
from dotenv import load_dotenv

# Load .env from project root
load_dotenv()  # looks for .env in the current working directory

AWS_ACCESS_KEY_ID = os.getenv("AWS_Acess_Key_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_Secret_Access_Key")
AWS_REGION = os.getenv("AWS_REGION", "us-east-1")

ATHENA_S3_OUTPUT = os.getenv("ATHENA_S3_OUTPUT")
ATHENA_DATABASE = os.getenv("ATHENA_DATABASE")

AWS_ACCESS_KEY_ID[:4], AWS_REGION, ATHENA_DATABASE


('AKIA', 'us-east-1', 's3_atmos_stclair')

### Load the data by creating boto session using awswrangler

In [2]:
import boto3
import awswrangler as wr
import pandas as pd

session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION,
)


### All the Tables in the Athena Database

In [3]:
df_tables = wr.catalog.tables(database=ATHENA_DATABASE, boto3_session=session)
table_names = df_tables["Table"].tolist()

print("Table names:", table_names)
print("Number of tables:", len(table_names))

Table names: ['achievement_user_completions', 'achievements', 'banner_clicks', 'bdg_user_completions', 'certification_users', 'comment_likes', 'community_comments', 'country_states', 'instances', 'instances_v', 'li_attempts', 'li_stats_agg_all', 'li_stats_agg_all_v', 'mi_user_mission_step_completions', 'offboarded_instances', 'poll_user_completions', 'rank_user_completions', 'rcs_historic_users', 'rcs_historic_users_v', 'rcs_lifetime_stats_by_month_v', 'rcs_user_lifetimes_no_stats_v', 'reward_prizes', 'reward_user_claims', 'reward_user_claims_v', 'rewards', 'sale_user_submissions', 'snippet_view_logs', 'snippets', 'survey_user_completions', 'test2_v', 'test_v', 'users']
Number of tables: 32


In [4]:
df_tables.head(32)

,Database,Table,Description,TableType,Columns,Partitions
0,s3_atmos_stclair,achievement_user_completions,,EXTERNAL_TABLE,"instance_id, generated_for_date, generated_ts,...",
1,s3_atmos_stclair,achievements,,EXTERNAL_TABLE,"instance_id, generated_for_date, generated_ts,...",
2,s3_atmos_stclair,banner_clicks,,EXTERNAL_TABLE,"instance_id, generated_for_date, generated_ts,...",
3,s3_atmos_stclair,bdg_user_completions,,EXTERNAL_TABLE,"instance_id, generated_for_date, generated_ts,...",
4,s3_atmos_stclair,certification_users,,EXTERNAL_TABLE,"instance_id, generated_for_date, generated_ts,...",
5,s3_atmos_stclair,comment_likes,,EXTERNAL_TABLE,"instance_id, generated_for_date, generated_ts,...",
6,s3_atmos_stclair,community_comments,,EXTERNAL_TABLE,"instance_id, generated_for_date, generated_ts,...",
7,s3_atmos_stclair,country_states,,EXTERNAL_TABLE,"country_short_code, country_name, state_name, ...",
8,s3_atmos_stclair,instances,,EXTERNAL_TABLE,"instance_id, client_name, site_name, site_url,...",
9,s3_atmos_stclair,instances_v,,VIRTUAL_VIEW,"instance_id, client_name, region_name, site_na...",


### Using Users table for EDA

### Dataset Overview & Data Types Distribution

In [5]:
query = "SELECT * FROM users"  

df_users = wr.athena.read_sql_query(
    sql=query,
    database=ATHENA_DATABASE,
    s3_output=ATHENA_S3_OUTPUT,
    boto3_session=session,
    ctas_approach=False,
    use_threads=True,
)



In [6]:
df_users.head()

,instance_id,generated_for_date,generated_ts,user_id,employee_number,account_created_ts,account_confirmed_ts,account_merged_ts,account_deleted_ts,account_disabled_ts,...,first_li_completion_ts,last_li_completion_ts,num_achievements,num_badges,num_user_referrals,num_sale_submissions,num_approved_sale_submissions,num_rejected_sale_submissions,num_pending_sale_submissions,num_connections
0,0E77CD92DB3440E20C2D118345397F9A,2026-02-08,2026-02-09 05:00:26,4,<NA>,2024-01-30 12:54:58,2024-01-30 12:54:58,NaT,NaT,NaT,...,2024-04-17 15:05:57,2024-04-17 15:05:57,0,1,0,<NA>,<NA>,<NA>,<NA>,0
1,823FB1CC552BEDC9B552EDA3335FBAE8,2025-07-31,2025-08-01 04:01:45,10003,<NA>,2025-03-24 11:07:34,2025-03-24 11:07:34,NaT,NaT,NaT,...,NaT,NaT,0,0,0,<NA>,<NA>,<NA>,<NA>,0
2,823FB1CC552BEDC9B552EDA3335FBAE8,2025-07-31,2025-08-01 04:01:45,4,<NA>,2024-01-30 12:54:58,2024-01-30 12:54:58,NaT,NaT,2025-03-25 13:43:03,...,2024-04-17 15:05:57,2024-04-17 15:05:57,0,1,0,<NA>,<NA>,<NA>,<NA>,0
3,0E81473CE89CCD67EB1F97B5C21FBE93,2025-10-13,2025-10-14 05:00:28,10005,<NA>,2024-10-16 09:01:00,2024-10-16 09:01:31,NaT,NaT,NaT,...,NaT,NaT,0,0,0,<NA>,<NA>,<NA>,<NA>,0
4,0E81473CE89CCD67EB1F97B5C21FBE93,2025-10-13,2025-10-14 05:00:28,10004,<NA>,2024-10-09 11:50:06,2024-10-09 11:50:41,NaT,NaT,NaT,...,NaT,NaT,0,0,0,<NA>,<NA>,<NA>,<NA>,0


In [7]:
len(df_users)

1183360

In [8]:
df_users.shape

(1183360, 82)

Count of Total users in Users table is 1180817 

**Full dataset scale: 1,180,817 rows × 82 columns – a comprehensive production user table capturing user profiles, lifecycle events, engagement metrics, and location data across multiple instances.**


In [9]:
df_users.dtypes.value_counts()

string[python]    44
Int32             18
datetime64[ns]    15
Int64              4
object             1
Name: count, dtype: int64

In [10]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1183360 entries, 0 to 1183359
Data columns (total 82 columns):
 #   Column                          Non-Null Count    Dtype         
---  ------                          --------------    -----         
 0   instance_id                     1183360 non-null  string        
 1   generated_for_date              1183360 non-null  object        
 2   generated_ts                    1183360 non-null  datetime64[ns]
 3   user_id                         1183360 non-null  string        
 4   employee_number                 148710 non-null   string        
 5   account_created_ts              1183353 non-null  datetime64[ns]
 6   account_confirmed_ts            1027550 non-null  datetime64[ns]
 7   account_merged_ts               7853 non-null     datetime64[ns]
 8   account_deleted_ts              545871 non-null   datetime64[ns]
 9   account_disabled_ts             385408 non-null   datetime64[ns]
 10  last_app_login_ts               469816 non

**Data Types Distribution:**
- **44 String columns (44)** – primarily user IDs (user_id, instance_id), statuses (account_status, verification_status), names (country_name, retailer, user_type), and location details (location_city, location_zipcode, etc.).
- **15 Datetime columns (15)** – timestamps for key events like account_created_ts, account_confirmed_ts, last_login_ts, last_online_ts, first_li_completion_ts, last_li_completion_ts, and verification dates (last_verified_ts, verification_expired_ts).
- **18 Int32 columns (18)** – flags and counts such as is_self_registration, is_quick_referral, is_opted_in_for_marketing_coms, num_li_completions, num_achievements, num_badges, num_user_referrals, and num_connections.
- **4 Int64 columns (4)**– high-range numerics like total_points_earned, total_spendable_points_earned, total_points_spent, and points_balance.
- **1 Object column (1)** – generated_for_date (likely date strings).

*This shows a rich dataset with user profiles, engagement metrics, and temporal activity data.*

In [11]:
# Show null counts and percentages for columns with missing data
null_counts = df_users.isnull().sum()
null_pct = (df_users.isnull().sum() / len(df_users)) * 100

null_summary = pd.DataFrame({
    'Null_Count': null_counts[null_counts > 0],
    'Null_Percentage': null_pct[null_counts > 0]
}).sort_values('Null_Count', ascending=False)

# Display ALL rows
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

null_summary

,Null_Count,Null_Percentage
account_merged_ts,1175507,99.336381
custom_field,1174423,99.244778
location_address_line2,1144141,96.685793
num_rejected_sale_submissions,1140631,96.389180
num_approved_sale_submissions,1140631,96.389180
num_pending_sale_submissions,1140631,96.389180
num_sale_submissions,1140631,96.389180
admin_auto_access_id,1134492,95.870403
admin_access_id,1134492,95.870403
admin_access_title,1134492,95.870403


In [12]:
# Show columns with NO null values
no_nulls = df_users.isnull().sum() == 0
print("Columns with NO null values:")
df_users.columns[no_nulls].tolist()

Columns with NO null values:


['instance_id',
 'generated_for_date',
 'generated_ts',
 'user_id',
 'account_status',
 'verification_status',
 'is_self_registration',
 'is_quick_referral',
 'timezone',
 'can_display_user_info',
 'total_points_earned',
 'total_points_spent',
 'points_balance',
 'num_li_completions',
 'num_unique_li_completions',
 'num_achievements',
 'num_badges',
 'num_user_referrals',
 'num_connections']

In [13]:
# Count and list categorical columns
cat_cols = df_users.select_dtypes(include=['object', 'string','category']).columns
print(f"Categorical columns: {len(cat_cols)}")
print(cat_cols.tolist())

# Count and list numerical columns  
num_cols = df_users.select_dtypes(include=['number', 'Int32','Int64']).columns
print(f"\n Numerical columns: {len(num_cols)}")
print(num_cols.tolist())

Categorical columns: 45
['instance_id', 'generated_for_date', 'user_id', 'employee_number', 'account_status', 'verification_status', 'user_type_id', 'user_type', 'hierarchy_title', 'retailer_id', 'retailer', 'location_country_code', 'location_state', 'location_id', 'location_title', 'location_code', 'location_longitude', 'location_latitude', 'location_city', 'location_zipcode', 'location_address_line1', 'location_address_line2', 'location_metadata_json', 'country_code', 'country_name', 'user_rank_id', 'user_rank_title', 'first_verification_method', 'referred_by_referral_code', 'referred_by_user_id', 'referred_by_first_name', 'referred_by_last_name', 'referred_by_user_type_title', 'referred_by_retailer_title', 'referred_by_location_title', 'timezone', 'preferred_language_id', 'custom_field', 'origin_auth_source_id', 'admin_access_id', 'admin_access_title', 'admin_auto_access_id', 'report_access_id', 'report_access_title', 'report_auto_access_id']

 Numerical columns: 22
['user_rank_numb

**Columns Distribution:**
- **45 Categorical columns** (IDs, statuses, names, locations, types)
- **22 Numerical columns** (points, counts, completions, metrics)
- **15 Datetime columns** (activity timestamps)
